In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from valhalla import Actor, get_config

In [15]:
CITY = "budapest"

In [2]:
config = get_config(
    tile_extract="../data/valhalla/valhalla_tiles.tar",
    verbose=True,
)

config["service_limits"]["isochrone"]["max_contours"] = 10
config["service_limits"]["isochrone"]["max_locations"] = 10_000
config["service_limits"]["isochrone"]["max_distance"] = 100_000

# instantiate Actor to load graph and call actions
actor = Actor(config)

2025/01/28 07:37:21.412073 [INFO] Tile extract successfully loaded with tile count: 255
2025/01/28 07:37:21.419486 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/01/28 07:37:21.419519 [WARN] Traffic tile extract could not be loaded


In [ ]:
stops = pd.read_csv(f"../data/stops/{CITY}/stops_15min.csv", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.to_crs(23700, inplace=True)
stops.head(3)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,parent_station,wheelchair_boarding,stop_direction,clust,max_distance,quant,geometry
0,008951,Béke tér,47.430019,19.215675,None,NaN,008951,2.0,-64.0,11,0.0,1,POINT (662691.105 231796.227)
1,F03807,Szent László utca,47.441814,19.207651,None,NaN,F03807,2.0,30.0,12,0.0,1,POINT (662083.110 233106.253)
2,F03851,Selmecbánya utca,47.438409,19.204790,None,NaN,F03851,2.0,29.0,13,0.0,1,POINT (661868.082 232727.272)


In [6]:
locations = []
for row in stops.itertuples():
    locations.append({"lon": row.stop_lon, "lat": row.stop_lat})

In [7]:
def get_isochrone(query: dict):
    result = {}
    isochrones = actor.isochrone(query)
    for contour_ix, isochrone in enumerate(isochrones["features"]):
        geom = isochrone["geometry"]["coordinates"]
        time = isochrone["properties"]["contour"]
        result[time] = Polygon(geom)
    return result


def get_walk_isochrones(location: dict[str, float], times: list[int] = [5, 10, 15, 20]):
    query = {
        "locations": [location],
        "costing": "pedestrian",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_bike_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "bicycle",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_car_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "auto;",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)

In [10]:
isochones_wide = []
isochones = []
for row in stops.itertuples():
    w = get_walk_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    b = get_bike_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    isochones_wide.append([row.stop_id, w[5], w[10], w[15], b[5], b[10], b[15]])
    isochones.append([row.stop_id, w[5], "walk", 5])
    isochones.append([row.stop_id, w[10], "walk", 10])
    isochones.append([row.stop_id, w[15], "walk", 15])
    isochones.append([row.stop_id, b[5], "bicycle", 5])
    isochones.append([row.stop_id, b[10], "bicycle", 10])
    isochones.append([row.stop_id, b[15], "bicycle", 15])
isochones_wide = pd.DataFrame.from_records(
    isochones_wide,
    columns=["stop_id", "walk_5", "walk_10", "walk_15", "bike_5", "bike_10", "bike_15"],
)
isochones = pd.DataFrame.from_records(
    isochones, columns=["stop_id", "geometry", "costing", "range"]
)

In [11]:
isochones.head(10)

,stop_id,geometry,costing,range
0,008951,"POLYGON ((19.218675 47.433216, 19.215675 47.43...",walk,5
1,008951,"POLYGON ((19.220675 47.436717, 19.213848 47.43...",walk,10
2,008951,"POLYGON ((19.220675 47.43934, 19.219675 47.439...",walk,15
3,008951,"POLYGON ((19.217675 47.440074, 19.209986 47.43...",bicycle,5
4,008951,"POLYGON ((19.215675 47.449231, 19.214675 47.44...",bicycle,10
5,008951,"POLYGON ((19.210675 47.456055, 19.209292 47.45...",bicycle,15
6,F03807,"POLYGON ((19.212651 47.443938, 19.211651 47.44...",walk,5
7,F03807,"POLYGON ((19.214651 47.447643, 19.210651 47.44...",walk,10
8,F03807,"POLYGON ((19.216651 47.449819, 19.215651 47.45...",walk,15
9,F03807,"POLYGON ((19.216651 47.449836, 19.214164 47.45...",bicycle,5


In [12]:
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

In [14]:
isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file("../output/{CITX/isochrones.geojson")